In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [23]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)

In [24]:
model = ChatOpenAI()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful chatbot capable of performing a variety of tasks on the data available"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [28]:
print(memory.load_memory_variables({}))

{'chat_history': []}


In [29]:
chain = RunnablePassthrough.assign(
    #chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    **{
        'chat_history': RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    }
) | prompt | model

In [31]:
inputs = {"input": "hi im bob"}
response = chain.invoke(inputs)
print(type(response))
print(response)

<class 'langchain_core.messages.ai.AIMessage'>
content='Hello Bob! How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 31, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-608e3a87-7709-4765-83eb-02823e2df383-0'


In [35]:
print(type(memory.chat_memory))
print(memory.chat_memory)

<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
Human: hi im bob
AI: Hello Bob! How can I assist you today?


In [33]:
memory.chat_memory.add_user_message("hi im bob")
memory.chat_memory.add_ai_message('Hello Bob! How can I assist you today?')

In [34]:
inputs = {"input": "what is my name"}
response = chain.invoke(inputs)
print(response)

content='Your name is Bob. How can I assist you further, Bob?' response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 53, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1e3c74cd-fe7c-447f-8d47-2c3c63a95af8-0'


In [39]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_history = ChatMessageHistory()
chat_history.add_user_message("hi im bob")
chat_history.add_ai_message('Hello Bob! How can I assist you today?')

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True, chat_memory=chat_history)

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

inputs = {"input": "what is my name?"}
response = chain.invoke(inputs)

print(response)

content='Your name is Bob, as you mentioned earlier. How can I assist you further, Bob?' response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 43, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8521df89-6c7d-497f-9f0a-f212c046e2e4-0'


In [40]:
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='hi im bob'), AIMessage(content='Hello Bob! How can I assist you today?')]}
